# LSTNet with Gluon via SageMaker
This notebook takes original code by Sina Alfrooze and adapts it to run via Amazon SageMaker

Original code repo: https://github.com/safrooze/LSTNet-Gluon


In [13]:
import os
import time
import boto3
import s3fs
import sagemaker
from sagemaker.mxnet import MXNet
from mxnet import gluon
from sagemaker import get_execution_role
import train as t

In [14]:
#!pip install mxnet-cu90 --pre
import mxnet as mx
mx.__version__

'1.2.0'

## S3 location
Let's start by specifying:
- The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these. Here we use the `get_execution_role` function to obtain the role arn which was specified when creating the notebook.

In [15]:
model_dir = './model/' # local model directory
bucket = 'spturner-sagemaker-ireland'
prefix = 'demos/lstnet-gluon/forecast-electricity'

sagemaker_session = sagemaker.Session()
role = get_execution_role()

s3_data_path = "{}/{}/data".format(bucket, prefix)
s3_output_path = "{}/{}/output".format(bucket, prefix)
print('Data location: %s'%s3_data_path)
print('Output location: %s'%s3_output_path)

Data location: spturner-sagemaker-ireland/demos/lstnet-gluon/forecast-electricity/data
Output location: spturner-sagemaker-ireland/demos/lstnet-gluon/forecast-electricity/output


# Download dataset and set path to file

In [16]:
#!wget https://github.com/laiguokun/multivariate-time-series-data/raw/master/electricity/electricity.txt.gz -P ./data/
#!gunzip ./data/electricity.txt.gz
train_data = './data/electricity.txt'

## Uploading the data
We use the sagemaker.Session.upload_data function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use this later when we start the training job.

In [17]:
inputs = sagemaker_session.upload_data(train_data, bucket=bucket, key_prefix=prefix)
inputs

## Display dataset via pandas


In [5]:
#display electricity.txt file
import pandas
df = pandas.read_csv(train_data, na_values=['?'], header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,311,312,313,314,315,316,317,318,319,320
0,14.0,69.0,234.0,415.0,215.0,1056.0,29.0,840.0,226.0,265.0,...,676.0,372.0,80100.0,4719.0,5002.0,48.0,38.0,1558.0,182.0,2162.0
1,18.0,92.0,312.0,556.0,292.0,1363.0,29.0,1102.0,271.0,340.0,...,805.0,452.0,95200.0,4643.0,6617.0,65.0,47.0,2177.0,253.0,2835.0
2,21.0,96.0,312.0,560.0,272.0,1240.0,29.0,1025.0,270.0,300.0,...,817.0,430.0,96600.0,4285.0,6571.0,64.0,43.0,2193.0,218.0,2764.0
3,20.0,92.0,312.0,443.0,213.0,845.0,24.0,833.0,179.0,211.0,...,801.0,291.0,94500.0,4222.0,6365.0,65.0,39.0,1315.0,195.0,2735.0
4,22.0,91.0,312.0,346.0,190.0,647.0,16.0,733.0,186.0,179.0,...,807.0,279.0,91300.0,4116.0,6298.0,75.0,40.0,1378.0,191.0,2721.0


# Train the model in the notebook (No SageMaker)

In [ ]:
t.train(train_data, './model/model.params')

Training Start
Epoch   0: loss 749.6
Epoch   1: loss 394.4
Epoch   2: loss 328.8
Epoch   3: loss 278.6
Epoch   4: loss 253.9
Epoch   5: loss 238.7
Epoch   6: loss 227.7
Epoch   7: loss 220.9
Epoch   8: loss 212.9


# Train model on SageMaker

In [ ]:
import train as t
hyperparameters={'batch_size': 100, 
                         'epochs': 10, 
                         'learning_rate': 0.1, 
                         'momentum': 0.9, 
                         'log_interval': 100})

t.train({'training': train_data},
    hyperparameters=hyper_parameters, 
    hosts=['local'], num_gpus=0)

# Saving the model 

In [ ]:
def save(net, model_dir):
    # save the model
    y = net(mx.sym.var('data'))
    y.save('%s/model.json' % model_dir)
    net.collect_params().save('%s/model.params' % model_dir)